<a href="https://colab.research.google.com/github/danylonepochatov/calculator/blob/main/Nepochatov_Danylo_10_2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [24]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [23]:
fahrenheit_to_celsius_multiplier = 5 / 9
fahrenheit_to_celsius_offset = 32

def read_fahrenheit_temperatures(filepath: str) -> list[float]:
    temperature = []


    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            fahrenheit = float(row[0])
            celsius = (fahrenheit - fahrenheit_to_celsius_offset) * fahrenheit_to_celsius_multiplier
            temperature.append(celsius)
    return temperature

def calculate_statistics(temperatures: list[float]) -> tuple[float, float, float]:

    average = sum(temperatures) / len(temperatures)
    minimun = min(temperatures)
    maximum = max(temperatures)
    return (average, minimun, maximum)

def print_statistics(average: float, minimum: float, maximum: float) -> None:

    print("Statistics:")
    print("Average: {average:.2f}°C")
    print("Minimum: {minimum:.2f}°C")
    print("Maximum: {maximum:.2f}°C")

def write_celcius_to_csv(filepath: str, temperatures: list[float]) -> None:
    with open(filepath, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])
        for temp in temperatures:
            writer.writerow([f"{temp:2f}"])


def process_temperature_data(input_path: str, output_path: str) -> None:

    celsius_temps = read_fahrenheit_temperatures(input_path)
    avg, min_temp, max_temp = calculate_statistics(celsius_temps)
    print_statistics(avg, min_temp, max_temp)
    write_celcius_to_csv(output_path, celsius_temps)

### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [28]:
import csv
from unittest.mock import mock_open, patch

class UnitTestCase(unittest.TestCase):

    def test_calculate_temperature_statistics(self):
        temperatures = [20.0, 22.0, 18.0]
        average, minimum, maximum = calculate_statistics(temperatures)
        self.assertAlmostEqual(average, 20.0)
        self.assertEqual(minimum, 18.0)
        self.assertEqual(maximum, 22.0)

    @patch("builtins.open", new_callable=mock_open, read_data="Temperature (F)\n78.8\n86.0")
    def test_read_temperatures_in_celsius_from_csv(self, mock_file):

        with patch("csv.reader", return_value=iter([["Temperature (F)"], ["78.8"], ["86.0"]])):
            result = read_fahrenheit_temperatures("dummy.csv")
        expected = [(78.8 - 32) * 5 / 9, (86.0 - 32) * 5 / 9]
        self.assertAlmostEqual(result[0], expected[0], places=2)
        self.assertAlmostEqual(result[1], expected[1], places=2)


class IntegrationTestCase(unittest.TestCase):

    @patch("builtins.open", new_callable=mock_open)
    def test_process_temperature_data(self, mock_file):

        mock_reader = iter([["Temperature (F)"], ["78.8"], ["86.0"]])


        with patch("csv.reader", return_value=mock_reader):
            with patch("csv.writer") as mock_writer_class:
                mock_writer = mock_writer_class.return_value
                process_temperature_data("input.csv", "output.csv")


                self.assertIn(
                    unittest.mock.call(["Temperature (C)"]),
                    mock_writer.writerow.call_args_list
                )

                self.assertEqual(mock_writer.writerow.call_count, 3)


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.014s

OK


Statistics:
Average: {average:.2f}°C
Minimum: {minimum:.2f}°C
Maximum: {maximum:.2f}°C
